In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from litequeue import SQLQueue

import sqlite3

In [3]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

In [4]:
for conn_type_kwargs in (
    {"filename_or_conn": sqlite3.connect(":memory:")},
    {"filename_or_conn": ":memory:"},
    {"memory": True},
):
    q = SQLQueue(**conn_type_kwargs)
    assert (
        q.conn.isolation_level is None
    ), f"Isolation level not set properly for connection '{conn_type_kwargs}'"

In [5]:
q = SQLQueue(sqlite3.connect(":memory:"))

q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

4

In [6]:
q.pop()

{'message': 'hello', 'message_id': '706201f6e89e7c81aa77e57d49396380'}

In [7]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fa29c5f3730>, items=[{'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '706201f6e89e7c81aa77e57d49396380',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': None,
  'message': 'world',
  'message_id': '78c5ac43ede9a8edbf88a9ff0300e568',
  'status': 0},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': None,
  'message': 'foo',
  'message_id': 'bf6a9c04d08637c62347d5406ec03fbd',
  'status': 0},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': None,
  'message': 'bar',
  'message_id': '09b99e844d29cf1511492697b26e6309',
  'status': 0}])


In [8]:
# pop remaining
for _ in range(3):
    q.pop()


assert q.pop() is None

In [9]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fa29c5f3730>, items=[{'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '706201f6e89e7c81aa77e57d49396380',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'world',
  'message_id': '78c5ac43ede9a8edbf88a9ff0300e568',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'foo',
  'message_id': 'bf6a9c04d08637c62347d5406ec03fbd',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'bar',
  'message_id': '09b99e844d29cf1511492697b26e6309',
  'status': 1}])


In [10]:
q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

8

In [11]:
task = q.pop()

assert task["message"] == "hello"

In [12]:
q.peek()

{'message': 'world',
 'message_id': '7d65a7f851364e85d805d6a30d5c3dbd',
 'status': 0,
 'in_time': 1617129949,
 'lock_time': None,
 'done_time': None}

In [13]:
# next one that is free
assert q.peek()["message"] == "world"

# status = 0 = free
assert q.peek()["status"] == 0

In [14]:
task["message"], task["message_id"]

('hello', '5b515520ff3525190e9c8d928c266dcf')

In [15]:
q.done(task["message_id"])

8

In [16]:
q.get(task["message_id"])

{'message': 'hello',
 'message_id': '5b515520ff3525190e9c8d928c266dcf',
 'status': 2,
 'in_time': 1617129949,
 'lock_time': 1617129949,
 'done_time': 1617129949}

In [17]:
already_done = q.get(task["message_id"])

# stauts = 2 = done
assert already_done["status"] == 2

in_time = already_done["in_time"]
lock_time = already_done["lock_time"]
done_time = already_done["done_time"]

assert done_time >= lock_time >= in_time
print(
    f"Task {already_done['message_id']} took {done_time - lock_time} seconds to get done and was in the queue for {done_time - in_time} seconds"
)

Task 5b515520ff3525190e9c8d928c266dcf took 0 seconds to get done and was in the queue for 0 seconds


In [18]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fa29c5f3730>, items=[{'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '706201f6e89e7c81aa77e57d49396380',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'world',
  'message_id': '78c5ac43ede9a8edbf88a9ff0300e568',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'foo',
  'message_id': 'bf6a9c04d08637c62347d5406ec03fbd',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'bar',
  'message_id': '09b99e844d29cf1511492697b26e6309',
  'status': 1},
 {'done_time': 1617129949,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '5b515520ff3525190e9c8d928c266dcf',
  'status': 2},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': None,
  'message': 'world',
  'message_id': '7d65

In [19]:
assert q.qsize() == 7

In [20]:
next_one_msg = q.peek()["message"]
next_one_id = q.peek()["message_id"]

task = q.pop()

assert task["message"] == next_one_msg
assert task["message_id"] == next_one_id

In [21]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fa29c5f3730>, items=[{'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '706201f6e89e7c81aa77e57d49396380',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'world',
  'message_id': '78c5ac43ede9a8edbf88a9ff0300e568',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'foo',
  'message_id': 'bf6a9c04d08637c62347d5406ec03fbd',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'bar',
  'message_id': '09b99e844d29cf1511492697b26e6309',
  'status': 1},
 {'done_time': 1617129949,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '5b515520ff3525190e9c8d928c266dcf',
  'status': 2},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129950,
  'message': 'world',
  'message_id':

In [22]:
q.prune()

In [23]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7fa29c5f3730>, items=[{'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'hello',
  'message_id': '706201f6e89e7c81aa77e57d49396380',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'world',
  'message_id': '78c5ac43ede9a8edbf88a9ff0300e568',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'foo',
  'message_id': 'bf6a9c04d08637c62347d5406ec03fbd',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129949,
  'message': 'bar',
  'message_id': '09b99e844d29cf1511492697b26e6309',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': 1617129950,
  'message': 'world',
  'message_id': '7d65a7f851364e85d805d6a30d5c3dbd',
  'status': 1},
 {'done_time': None,
  'in_time': 1617129949,
  'lock_time': None,
  'message': 'foo',
  'message_id': '84002c491914

In [24]:
from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [25]:
q = SQLQueue(":memory:", maxsize=50)

In [26]:
for i in range(50):

    q.put(random_string(20))

In [27]:
assert q.qsize() == 50

Make sure an error is raised when the queue has reached its size limit

In [28]:
import sqlite3

try:
    q.put(random_string(20))
except sqlite3.IntegrityError:  # max len reached
    assert q.full() == True
    print("test pass")

test pass


In [29]:
q.pop()

{'message': 'nhqzsrzkgoqenpbrjlex',
 'message_id': '78c2d613848fd2bdcac9d5087eaa2e18'}

In [30]:
assert q.full() == False

In [31]:
q.put("hello")

51

In [32]:
q.empty()

False

In [33]:
assert q.empty() == False

q2 = SQLQueue(":memory:")

assert q2.empty() == True

**Random benchmarks**

Inserting 100 items in the queue. Since it will run many times, the queue will end up having a lot more than 100 items

In [34]:
import gc

In-memory SQL queue

In [35]:
q = SQLQueue(":memory:", maxsize=None)

In [36]:
gc.collect()

264

In [37]:
%%timeit -n10000 -r7

q.put(random_string(20))

43.7 µs ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [38]:
q.qsize()

70000

Standard python queue.

In [39]:
from queue import Queue

In [40]:
q = Queue()

In [41]:
gc.collect()

113

In [42]:
%%timeit -n10000 -r7

q.put(random_string(20))

20.3 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Persistent SQL queue

In [43]:
q = SQLQueue("test.queue", maxsize=None)

In [44]:
gc.collect()

69

In [45]:
%%timeit -n10000 -r7

q.put(random_string(20))

156 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [46]:
!du -sh test.queue*

9.1M	test.queue
32K	test.queue-shm
5.0M	test.queue-wal


In [47]:
!rm test.queue*

In [48]:
assert q.conn.isolation_level is None

Creating and removing tasks

In [49]:
q = Queue()

In [50]:
gc.collect()

135

In [51]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

q.get()

q.task_done()

25.4 µs ± 2.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [52]:
q = SQLQueue(":memory:", maxsize=None)

In [53]:
gc.collect()

69

In [54]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

task = q.pop()

q.done(task["message_id"])

94.9 µs ± 8.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
